# PyOPV

## Installation

## Example usage

### Importing

In [1]:
import sys; sys.path.append('..') # add parent directory to path (for testing)
import pyopv

## Getting latest DICOM Standards

In [4]:
from pyopv import get_dicom_standard # if you run this cell, you will get the latest SAP DICOM standard
get_dicom_standard() # if you run this cell, you will get the latest SAP DICOM standard and writes in a csv file in the same directory as this notebook

Up to date DICOM standard json files successfully extracted.
Up to date DICOM attributes successfully extracted for CIOD: ophthalmic-visual-field-static-perimetry-measurements.
Up to date DICOM attributes successfully written to /ophthalmic-visual-field-static-perimetry-measurements.csv.
Headers:
           tag                        name                   keyword  \
1  (0008,0005)      Specific Character Set      SpecificCharacterSet   
5  (0008,0012)      Instance Creation Date      InstanceCreationDate   
6  (0008,0013)      Instance Creation Time      InstanceCreationTime   
7  (0008,0014)        Instance Creator UID        InstanceCreatorUID   
8  (0008,0015)  Instance Coercion DateTime  InstanceCoercionDateTime   

  valueRepresentation valueMultiplicity retired        id  
1                  CS               1-n       N  00080005  
5                  DA                 1       N  00080012  
6                  TM                 1       N  00080013  
7                  UI        

### Checking missing tags

A single file

In [5]:
from pyopv import OPVDicom

file_path = '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20220908_OD_OPV.dcm'
m_opvdicom = pyopv.read_dicom(file_path)
missing_tags_df, incorrect_tags_df= m_opvdicom.check_dicom_compliance() # check if the dicom file is compliant with the standard
display(missing_tags_df)
display(incorrect_tags_df)

,name,description,type,vm,vr,tag
0,DataSetName,The name assigned to the data set.,1,1.0,LO,"(0024, 0306)"
1,DataSetVersion,The software version identifier assigned to th...,1,1.0,LO,"(0024, 0307)"
2,DataSetSource,Source of the data set e.g. the name of the ma...,1,1.0,LO,"(0024, 0308)"
3,DataSetDescription,Description of the data set.,3,1.0,LO,"(0024, 0309)"
4,AlgorithmFamilyCodeSequence,The family of algorithm(s) that best describes...,1,1.0,SQ,"(0066, 002F)"
...,...,...,...,...,...,...
73,PupilSize,The horizontal diameter measurement of the pup...,2,1.0,FD,"(0046, 0044)"
74,PupilDilated,The patient's pupils were pharmacologically di...,2,1.0,CS,"(0022, 000D)"
75,IntraOcularPressure,Value of intraocular pressure in mmHg.,3,1.0,FL,"(0022, 000B)"
76,VisualAcuityMeasurementSequence,Measurements of a patient's visual acuity.,3,1.0,SQ,"(0024, 0110)"


""


Entire directory

In [6]:
m_opvdicoms = pyopv.read_dicom_directory('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files', file_extension='dcm')
missingtags_df = m_opvdicoms.check_dicom_compliance()
missingtags_df.head(5)



/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/PyOPV Paper/py-opv/examples/../pyopv/opvdicom/__init__.py:303: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_tags_df = pd.concat([missing_tags_df, pd.DataFrame({


,File Name,Missing tags Count / Missing DICOM Meta Information Header,Number of Missing Required Tags,Percentage of Missing Required Tags
0,ops_opv.dcm,93,75,73.53
1,test_716621_opv_od.dcm,82,63,61.76
2,SD1031_20230914_OS_OPV.dcm,81,63,61.76
3,SD1031_20230914_OD_OPV.dcm,81,63,61.76
4,SD1031_20220908_OD_OPV.dcm,78,62,60.78


## Converting OPV DICOM to Pandas DataFrame

*#* This snippet converts the entire dicom file into a very long pandas DataFrame

In [7]:
from pyopv import OPVDicom

mopv = pyopv.read_dicom('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm') 
mopv_df = mopv.to_pandas()
mopv_df.head(2)


name,Specific Character Set,Instance Creation Date,Instance Creation Time,SOP Class UID,SOP Instance UID,Study Date,Study Time,Accession Number,Modality,Manufacturer,...,Performed Protocol Code Sequence[1].Protocol Context Sequence[0].Concept Name Code Sequence,Performed Protocol Code Sequence[1].Protocol Context Sequence[0].Concept Name Code Sequence[0].Code Value,Performed Protocol Code Sequence[1].Protocol Context Sequence[0].Concept Name Code Sequence[0].Coding Scheme Designator,Performed Protocol Code Sequence[1].Protocol Context Sequence[0].Concept Name Code Sequence[0].Code Meaning,Private Creator,Private tag data,Private Creator,Private tag data,Private tag data,Private tag data
value,ISO_IR 192,20231115,115903.087,1.2.840.10008.5.1.4.1.1.80.1,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,20230914,103811.519,,OPV,Carl Zeiss Meditec,...,"[[(0008, 0100) Code Value ...",R-408C3,SRT,Diagnostic,99CZM_Hfa_OphthalmicVisualFieldStaticPerimetry,Central,99CZM_NIM_INTERNAL_01,HfaOphthalmicVisualFieldStaticPerimetryMeasure...,2.13,HfaVisualFieldStaticPerimetryTestResults\HfaVi...


## Bulk Data


In [2]:
from pyopv import OPVDicomSet

mopv_set = pyopv.read_dicom_directory('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files', file_extension='dcm')
result_df, error_df = mopv_set.to_pandas()
display(result_df)
display(error_df)

name,Visual Field Test Point Sequence[56].Visual Field Test Point Normals Sequence,Visual Field Test Point Sequence[29].Visual Field Test Point Normals Sequence,Visual Field Test Point Sequence[30].Stimulus Results,Presented Visual Stimuli Data Flag,Visual Field Test Point Sequence[15].Visual Field Test Point Normals Sequence[0].Generalized Defect Corrected Sensitivity Deviation Value,Visual Field Test Point Sequence[48].Visual Field Test Point X-Coordinate,Visual Field Test Point Sequence[53].Sensitivity Value,Visual Field Test Point Sequence[1].Visual Field Test Point Normals Sequence[0].Generalized Defect Corrected Sensitivity Deviation Flag,Visual Field Test Point Sequence[65].Stimulus Results,Stimulus Color Code Sequence[0].Coding Scheme Designator,...,Visual Field Test Point Sequence[56].Visual Field Test Point Normals Sequence[0].Generalized Defect Corrected Sensitivity Deviation Probability Value,Visual Field Test Point Sequence[33].Visual Field Test Point Normals Sequence[0].Generalized Defect Corrected Sensitivity Deviation Probability Value,Visual Field Test Point Sequence[25].Visual Field Test Point Normals Sequence,Referring Physician's Name,Visual Field Test Point Sequence[36].Visual Field Test Point Normals Sequence[0].Age Corrected Sensitivity Deviation Value,Referenced Instance Sequence[0].Referenced SOP Instance UID,Visual Field Test Point Sequence[13].Visual Field Test Point Y-Coordinate,Visual Field Test Point Sequence[58].Visual Field Test Point Normals Sequence,Visual Field Test Point Sequence[45].Visual Field Test Point Normals Sequence[0].Generalized Defect Corrected Sensitivity Deviation Probability Value,Blind Spot Localized
0,NaN,NaN,SEEN,NaN,NaN,-3.0,0.0,NaN,NaN,SCT,...,NaN,NaN,NaN,(),NaN,NaN,-4.0,NaN,NaN,NO
1,NaN,"[[(0024, 0092) Age Corrected Sensitivity Devia...",SEEN,NO,-9.415,3.0,NaN,YES,NaN,SCT,...,NaN,0.5,"[[(0024, 0092) Age Corrected Sensitivity Devia...",(),-28.483,NaN,3.0,NaN,0.5,YES
2,"[[(0024, 0092) Age Corrected Sensitivity Devia...","[[(0024, 0092) Age Corrected Sensitivity Devia...",SEEN,NO,0.0,-1.0,27.0,YES,SEEN,SRT,...,0.0,0.0,"[[(0024, 0092) Age Corrected Sensitivity Devia...",(),-8.0,1.2.276.0.75.2.2.30.2.3.230914105829918.501269...,5.0,"[[(0024, 0092) Age Corrected Sensitivity Devia...",5.0,NO
3,"[[(0024, 0092) Age Corrected Sensitivity Devia...","[[(0024, 0092) Age Corrected Sensitivity Devia...",SEEN,NO,0.0,1.0,29.0,YES,SEEN,SRT,...,5.0,0.0,"[[(0024, 0092) Age Corrected Sensitivity Devia...",(),-4.0,1.2.276.0.75.2.2.30.2.3.230914104727554.501269...,5.0,"[[(0024, 0092) Age Corrected Sensitivity Devia...",0.0,NO
4,NaN,"[[(0024, 0092) Age Corrected Sensitivity Devia...",SEEN,NO,-4.0,9.0,32.0,YES,NaN,SRT,...,NaN,0.0,"[[(0024, 0092) Age Corrected Sensitivity Devia...",(),-2.0,1.2.276.0.75.2.2.30.2.3.220908091316457.501269...,9.0,NaN,0.5,YES


,file_name,error


# JSON conversion
* this snippet will be updated in future versions to accomodate FHIR IG because that IG is not currently available. 

In [9]:
import pydicom

file_path =  '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm'  # add file path here e.g. data/MY_OPV_FILE.dcm
ds = pydicom.dcmread(file_path)
json = ds.to_json_dict()
json['00080080']['Value']

['UCSD Ophthalmology Research']

In [10]:
# get nested values
json = ds.to_json_dict()
points = json['00240089']['Value'][0]['00240097']
points

{'vr': 'SQ',
 'Value': [{'00240092': {'vr': 'FL', 'Value': [-23.0]},
   '00240100': {'vr': 'FL', 'Value': [1.0]},
   '00240102': {'vr': 'CS', 'Value': ['YES']},
   '00240103': {'vr': 'FL', 'Value': [-20.0]},
   '00240104': {'vr': 'FL', 'Value': [1.0]}}]}

### Getting pointwise data

Single file

In [11]:
m_opvdicom = pyopv.read_dicom('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm')

pointwise_data = m_opvdicom.pointwise_to_pandas()
pointwise_data.head(5)

,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-1.0,9.0,SEEN,7.0,-23.0,1.0,YES,-20.0,1.0
1,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,1.0,9.0,SEEN,20.0,-9.0,1.0,YES,-6.0,2.0
2,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-5.0,7.0,SEEN,26.0,-4.0,5.0,YES,-1.0,0.0
3,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-3.0,7.0,SEEN,25.0,-6.0,1.0,YES,-2.0,0.0
4,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-1.0,7.0,SEEN,23.0,-7.0,1.0,YES,-4.0,2.0


Entire directory

In [12]:
m_opvdicoms = pyopv.read_dicom_directory('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files', file_extension='dcm')

pointwise_data, error_df = m_opvdicoms.pointwise_to_pandas()
pointwise_data.head(3)

,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,3.0,3.0,SEEN,0.0,-30.4,0.5,YES,-9.415,0.5
1,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,9.0,3.0,SEEN,0.0,-30.511,0.5,YES,-9.526,0.5
2,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,21.0,3.0,SEEN,0.0,-29.733,0.5,YES,-8.748,0.5


Writing pointwise data into JSON
* single file

In [13]:
m_opvdicom = pyopv.read_dicom('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm')

pointwise_data = m_opvdicom.pointwise_to_nested_json()
pointwise_data['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.231115115903087.345048637379.415558310'][0]

{'study_instance_uid': '1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084',
 'x_coords': -1.0,
 'y_coords': 9.0,
 'stimulus_result': 'SEEN',
 'sensitivity_values': 7.0,
 'age_corrected_sensitivity_deviation_values': -23.0,
 'age_corrected_sensitivity_deviation_probability_values': 1.0,
 'generalized_defect_corrected_sensitivity_deviation_flag': 'YES',
 'generalized_defect_corrected_sensitivity_values': -20.0,
 'generalized_defect_corrected_sensitivity_probability_values': 1.0}

* Bulk data

In [14]:
from pyopv import OPVDicomSet

directory_path = '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files'
m_opvdicoms = pyopv.read_dicom_directory(directory_path, file_extension='dcm')
nested_json = m_opvdicoms.opvdicoms_pointwise_to_nested_json()
nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.221018115130492.345048637379.430073977'][0]

Error processing file ops_opv.dcm: (0024, 0097)


{'study_instance_uid': '1.2.276.0.75.2.2.30.2.1.20220908090351.860.15084',
 'x_coords': -9.0,
 'y_coords': 21.0,
 'stimulus_result': 'SEEN',
 'sensitivity_values': 27.0,
 'age_corrected_sensitivity_deviation_values': 1.0,
 'age_corrected_sensitivity_deviation_probability_values': 0.0,
 'generalized_defect_corrected_sensitivity_deviation_flag': 'YES',
 'generalized_defect_corrected_sensitivity_values': 0.0,
 'generalized_defect_corrected_sensitivity_probability_values': 0.0}

# Example conversion of a JSON resource to pandas DataFrame

In [15]:
import pandas as pd
pd.json_normalize(nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.221018115130492.345048637379.430073977'], sep='_').head(3)

,study_instance_uid,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,1.2.276.0.75.2.2.30.2.1.20220908090351.860.15084,-9.0,21.0,SEEN,27.0,1.0,0.0,YES,0.0,0.0
1,1.2.276.0.75.2.2.30.2.1.20220908090351.860.15084,-3.0,21.0,SEEN,24.0,-2.0,0.0,YES,-4.0,0.0
2,1.2.276.0.75.2.2.30.2.1.20220908090351.860.15084,3.0,21.0,SEEN,24.0,-2.0,0.0,YES,-4.0,0.0
